In [ ]:
%matplotlib inline
from IPython.core.pylabtools import figsize
from IPython.display import YouTubeVideo

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits, make_moons, make_classification, make_blobs
from sklearn.metrics import roc_curve, auc
from sklearn.utils import shuffle

# Lecture 11: Support Vector Machines: SVM

We will discuss an approach for classification developed outside of the Statistical framework developed in the 90's by the computer science community, called SVM. SVM are often considered one of the best "out of the box" classifiers.

SVM is a generalisation of a simple and intuitive classifier called the *marximal margin classifier* which we talk about first.

## Maximal Margin Classifier

Before getting into this, we need to discuss and define a hyperplane and the optimal seperating hyperplane.

### Hyperplane?

In a $p$-dimensional setting we define a hyperplane as:

$$
\beta_0 + \beta_1 X_1  + \beta_2 X_2  + \cdots  + \beta_p X_p = 0
$$

in the sense that if a point $X=(X_1,\cdots,X_p)^{T}$ in $p$ dimensions satisfies the above equation, then it lies on this hyperplane. In the $p=3$ setting the hyperplane is 2-dimensional:

![](http://i.stack.imgur.com/zeRTm.png)

And if $X$ satisfies $\beta_0 + \beta_1 X_1  + \beta_2 X_2  + \cdots  + \beta_p X_p > 0$ then it tells us that $X$ is on one side of the hyperplane and if $>0$ then on the other. So the hyperplane divides the space into two halves and one can tell which side a point lies on by this sign calculation.

### Seperating Hyperplane

Suppose that it is possible to construct a hyperplane that separates the training observations perfectly according to their class labels.  Below we see examples of such hyperplanes:
![](http://upload.wikimedia.org/wikipedia/commons/2/20/SVM_Example_of_Hyperplanes.png)
If a separating hyperplane exists, we can use it to construct a very natural classifier: a test observation is assigned a class depending on which side of the hyperplane it is located. And if our data can be perfectly separated using a hyperplane, then there will in fact exist an infinite number of such hyperplanes.  But how do we decide  which of these infinite possible separating hyperplanes is a reasonable choice?

### Maximal Margin

A natural choice is the *maximal margin hyperplane* which is the separating hyperplane that is farthest from the training observations.  That is, we can compute the (perpendicular) distance from each training observation to a given seperating hyperlane; the smallest such distance is the minimal distance from the observations to the hyperplane, and is known as the margin. The maximal margin hyperplane is the separating hyperplane for which the margin is largest. 

We  hope that a classifier that has a large margin on the training data will also have a large margin on the test data, and hence will hope to classify the test observations correctly.

In [ ]:
# we create 40 separable points
np.random.seed(0)
X = np.r_[np.random.randn(20, 2) - [2, 2], np.random.randn(20, 2) + [2, 2]]
Y = [0] * 20 + [1] * 20

# fit the model
clf = svm.SVC(kernel='linear')
clf.fit(X, Y)

# get the separating hyperplane
w = clf.coef_[0]
a = -w[0] / w[1]
xx = np.linspace(-5, 5)
yy = a * xx - (clf.intercept_[0]) / w[1]

# plot the parallels to the separating hyperplane that pass through the
# support vectors
b = clf.support_vectors_[0]
yy_down = a * xx + (b[1] - a * b[0])
b = clf.support_vectors_[-1]
yy_up = a * xx + (b[1] - a * b[0])

# plot the line, the points, and the nearest vectors to the plane
figsize(11, 9)
plt.plot(xx, yy, 'k-')
plt.plot(xx, yy_down, 'k--')
plt.plot(xx, yy_up, 'k--')

plt.scatter(clf.support_vectors_[:, 0], 
            clf.support_vectors_[:, 1],
            s=200, facecolors='none')
plt.scatter(X[:, 0], X[:, 1], c=Y, 
            s=100, cmap=plt.cm.Paired)

plt.axis('tight')
plt.show()

### Construction of the Maximal Margin Classifier

So how do we contruct these margins given $n$ training observations $x_1,\ldots,x_n \in \mathbb{R}^p$ and labels $y_1,\ldots,y_n \in \{-1, 1\}$? We do the following:

* $ \max_{\beta_0,\ldots,\beta_p} M$
* subject to $\sum_{j=1}^{p} \beta_j^{2} = 1$ so that
* $y_i(\beta_0 + \sum_{j=1}^{p} \beta_j x_{ij}) \ge M$ for all $i=1,\ldots,n$.

The last constraint guarantees that each observation will be on the correct side of the hyperplane plus a margin, provided $M$ is positive.  

The second constraint is a constraint on the hyperplane it constraints the perpendicular distance from the $i$th observation to the hyperplane. So together these constraints ensure that each observation is on the correct side of the hyperplane and at least a distance $M$ from the hyperplane. The problem then chooses $\beta_0,\ldots,\beta_p$ to maximise $M$ and there are efficient ways of solving this.

All is good and well if the is a a separating hyperplane. However in many if not all cases no separating hyperplane exists, and so there is no maximal margin classifier. So we need to extend the concept of a seperating hyperplane  in order to develop a hyperplane that *almost* serperates the classes which is our next section.

## Support Vector Classifiers

A classifier based on a separating hyperplane will necessarily perfectly classify all of the training observations; this can lead to sensitivity to individual observations. A single outlier can cause a dramatic change in the maximal margin hyperplane. which is not a good thing and suggests overfitting. So for these two reason, we consider a classifier based on a hyperplane that does *not* perfectly seperate the two classes. In other words it could be worthwhile to  misclassify a few training observations in order to do a better job in classifying the remaining observations.

The *support vector classifer* does just this. The hyperplane is chosen to correctly separate most of the training observations into the two classes, but may misclassify a few observations. It is the solution to the optimization problem:

* $ \max_{\beta_0,\ldots,\beta_p, \epsilon_1,\ldots,\epsilon_n} M$
* subject to $\sum_{j=1}^{p} \beta_j^{2} = 1$ so that
* $y_i(\beta_0 + \sum_{j=1}^{p} \beta_j x_{ij}) \ge M(1-\epsilon_i)$ for all $i=1,\ldots,n$
* where $\epsilon_i \ge 0$ and $\sum_{i=1}^{n} \epsilon_i \le C$.

Here $C$ is a non-negative nonnegative tuning parameter. As before $M$ is the width of the margin we seek to make this quantity as large as possible. The $\epsilon_i$ are slack variables that  allow individual observations to be on the wrong side of the margin or hyperplane. When $\epsilon_i = 0$ then the $i$th observation is on the correct side of the margin. If $\epsilon_i > 0$ then the $i$th observation is on the wrong side of the margin. If $\epsilon_i > 1$ then it is on the wrong side of the hyperplane.

The tuning parameter $C$ acts like a *budget* for the amount the margin can be violated by the $n$ observations. When $C=0$ then there is no budget and all the $\epsilon_i$ must be 0. As $C$ increases we become more tolerant of the violations to the margin and the margin will widen.

In practice $C$ is is generally chosen via cross-validation. As with the tuning parameters that we have seen in the lectures before, $C$ controls the bias-variance trade-off of the statistical learning techniques. When $C$ is small we have a tight fit to the data so a low bias high variance situation. When $C$ is larger the margin is wider and this leads to higher bias but a lower variance.

Note that Scikit-Learn uses [LibSVM](http://www.csie.ntu.edu.tw/~cjlin/libsvm/) an external library for solving these problems and this library solves a minimisation problem. In which case for our $C$ above is actually $1/C$ in the Scikit parameter:

In [ ]:
X, Y = make_classification(n_features=2, n_redundant=0, 
                           n_informative=2, n_clusters_per_class=1)
X += 2 * np.random.uniform(size=X.shape)

# figure number
fignum = 1

# fit the model
for name, penalty in (('reg C large', 1/50.), ('unreg C small', 1/.5)):

    clf = svm.SVC(kernel='linear', C=penalty)
    clf.fit(X, Y)

    # get the separating hyperplane
    w = clf.coef_[0]
    a = -w[0] / w[1]
    xx = np.linspace(-5, 5)
    yy = a * xx - (clf.intercept_[0]) / w[1]

    # plot the parallels to the separating hyperplane that pass through the
    # support vectors
    margin = 1 / np.sqrt(np.sum(clf.coef_ ** 2))
    yy_down = yy + a * margin
    yy_up = yy - a * margin

    # plot the line, the points, and the nearest vectors to the plane
    plt.figure(fignum, figsize=(11, 9))
    plt.clf()
    plt.plot(xx, yy, 'k-')
    plt.plot(xx, yy_down, 'k--')
    plt.plot(xx, yy_up, 'k--')

    plt.scatter(clf.support_vectors_[:, 0], 
                clf.support_vectors_[:, 1], s=200,
                facecolors='none', zorder=10)
    plt.scatter(X[:, 0], X[:, 1], c=Y, zorder=10, s=80, cmap=plt.cm.Paired)

    plt.axis('tight')

    plt.xticks(())
    plt.yticks(())
    plt.title(name)
    fignum = fignum + 1

plt.show()

### Interesting property

It turns out that  only observations that either lie on the margin or that violate the margin will affect the hyperplane and hence the classifer we get. In other words, an observation that lies strictly on the correct side of the margin does *not* affect the support vector classifier! Changing the position of that observation would not change the classifier at all, provided that its position remains on the correct side of the margin. 

Observations that lie directly on the margin, or on the wrong side of the margin for their class are known as *support vectors*. These observations *do* affect the classifier.

This fact does not violate the bias-variance trade-off asserted by the parameter $C$.  When the tuning parameter $C$ is large, then the margin is wide, many observations  violate the margin, and so there are many support vectors.  In this case, many observations are involved in determining the hyperplane. This classifier has low variance but potentially high bias. 

On the other hand when $C$ is small, then there will be fewer support vectors and so the classifier will have low bias but high variance.

## Support Vector Machines

Support vector classifiers  are a  natural approach for classification in the two-class setting if the boundary between the two classes is linear. In practice we get non-linear boundaries as well.

We have been in this situation before when looking at linear regression. We saw that the performance of linear regression can suffer when there is a non-linear  relationship between the predictors and the outcome. In that case we considered enlarging the feature space using function of the predictors, such as quadratic and cubic terms in order to address this non-linearity.

In the case of the support vector classifier, we could address the problem of possibly non-linear boundaries between classes in a similar way, by enlarging the  the feature space using quadratic, cubic, and even higher-order polynomial functions of the predictors.

Why does this lead to a non-linear boundary? Perhaps this video will make it clear:

In [ ]:
YouTubeVideo('3liCbRZPrZA', start=2, width=600, height=500)

It is not hard to see that there are many possible ways to enlarge the feature space, and that unless we are careful, we could end up with a huge number of features. Then computations would become unmanageable. The support vector machine, which we present next, allows us to enlarge the feature space used by the support vector classifier in a way that leads to efficient computations.

## The Support Vector Machine

The support vector machine (SVM) is an extension of the support vector classifier that results from enlarging the feature space in a specific way, by using *kernels*. The main idea is as above. The kernel approach is simply an efficient computational approach for enacting this idea.

We did not go into the detail of how solve the maximisation problem but it turns out that  the solution to the support vector classifier problem involves only the *inner products* of the observations rather than the observations themselves. The inner product of two observations $x_i$ and $x_{i'}$ is:

$$
\langle x_i, x_{i'} \rangle = \sum_{j=1}^{p}x_{ij}x_{i'j}.
$$

It can be shown that:

* The linear support vector classifier can be represented as $$ f(x) = \beta_0 + \sum_{i=1}^{n} \alpha_i \langle x, x_{i} \rangle,$$ where there are $n$ parameters $\alpha_i$ for $i=1,\ldots,n$, one  per training observation.
* To estimate the parameters $\alpha_i$ and $\beta_0$ all we need are the $\binom{n}{2}$ inner products between all pairs of training observations.

Notice that in order to evaluate the function $f(x)$ we need to compute the inner product  between the new point $x$ and each of the training points $x_i$. However it turns out that $\alpha_i$ is non-zero for the support vectors in the solution: if a training observation is not a support vector then its $\alpha_i = 0$. So if $\cal{S}$ is the collection of indices of these support points, we can rewrite the solution function   as:
$$
f(x) = \beta_0 + \sum_{i\in \cal{S}} \alpha_i \langle x, x_{i} \rangle
$$
which involves far fewer terms.

Now suppose every time the inner product appears we replace it with a *generalisation* of the innner product of the form 

$$K(x_i,x_{i'}),$$

where $K$ is some function that we call the *kernel*. A kernel is a function that quantifies the similarity of two observations. For example if we take

$$K(x_i,x_{i'}) = \sum_{j=1}^{p}x_{ij}x_{i'j},$$

we would just get back the  the support vector classifier. This is a *linear* kernel since the support vector classifier is linear in the features; the linear kernel essentially quantifies the similarity of a pair of observations using Pearson (standard) correlation. But one could instead choose another form for the kernel like:

$$K(x_i,x_{i'}) = (1 + \sum_{j=1}^{p}x_{ij}x_{i'j})^d.$$

This is a *polynomial kernel* of degree $d > 0$. When the support vector classifier is combined with such a non-linear kernel the resulting classifier is called a *support vector machine*. Another popular kernel is the *radial kernel* given by:

$$K(x_i,x_{i'}) = \exp (- \gamma \sum_{j=1}^{p}(x_{ij} - x_{i'j})^2 ).$$

In [ ]:
X, Y = make_moons(noise=0.3)

# figure number
fignum = 1

# fit the model
for kernel in ('linear', 'poly', 'rbf'):
    clf = svm.SVC(kernel=kernel, gamma=2)
    clf.fit(X, Y)

    # plot the line, the points, and the nearest vectors to the plane
    plt.figure(fignum, figsize=(11, 9))
    plt.clf()

    plt.scatter(clf.support_vectors_[:, 0], clf.support_vectors_[:, 1],
               facecolors='none', zorder=10, s=200)
    plt.scatter(X[:, 0], X[:, 1], c=Y, zorder=10, s=80, cmap=plt.cm.Paired)

    plt.axis('tight')
    x_min = -2
    x_max = 2
    y_min = -2
    y_max = 2

    XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]
    Z = clf.decision_function(np.c_[XX.ravel(), YY.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(XX.shape)
    plt.figure(fignum, figsize=(11, 9))
    plt.contour(XX, YY, Z, colors=['k', 'k', 'k'], linestyles=['--', '-', '--'],
               levels=[-.5, 0, .5])

    plt.xlim(x_min, x_max)
    plt.ylim(y_min, y_max)

    plt.xticks(())
    plt.yticks(())
    fignum = fignum + 1
plt.show()

What is the advantage of using a kernel rather than simply enlarging the feature space using functions of the original features? One advantage is computational, and it amounts to the fact that using kernels one needs only compute $K(x_i,x_{i'})$ for all  $\binom{n}{2}$ distinct pairs $i, i'$. This can be done without explicitly working in the enlarged feature space. This is important  because in many applications of SVMs the enlarged feature space is so large that computations are not possible.

## SVMs with More than Two Classes

 How can we extend SVMs to the more general case where we have some arbitrary number of classes? It turns out that the concept of separating hyperplanes upon which SVMs are based does not lend itself naturally to more than two classes.  There are a number of  proposals for extending SVMs to the $K$-class case and the two most popular are discussed now.
 
### One-Versus-One Classification
 
A *one-versus-one* or *all-pairs* approach constructs  $\binom{K}{2}$ SVMs each of which compares a pair of classes. We classify a test observation using each of the $\binom{K}{2}$ classifiers and we tally the number of times that the test is assigned to each of the $K$ classes. The final classification is performed by  assigning the test observation to the class to which it was most frequently assigned.

### One-Versus-All Classification

The *one-versus-all* approach is an alternative procedure where we fit $K$ SVMs each time comparing one of the $K$ classes to the remaining $K-1$ classes. We then assign a test observation to the class with the highest level of confidence that the test observation belongs to this class rather than any other one.

## SVM vs LDA 
We now investigate how an SVM compares to LDA. We first fit LDA and the support vector classifier to the training data.
Note that the support vector classifier is equivalent to a SVM using a polynomial of degree $d=1$.  We can then plot a curve to represent training error rates known as ROC curves. This curve is obtained by forming predictions and computing the false positive and true positive rates $\hat{f}(X) < t$ and $\hat{f}(X) \ge t$ for a range of values of $t$. An optimal classifier will hug the top left corner of the ROC plot (see Lecture 5).

In [ ]:
X, Y = make_moons(noise=0.4)

# shuffle and split training and test sets
X, Y = shuffle(X, Y)
half = 50
X_train, X_test = X[:half], X[half:]
y_train, y_test = Y[:half], Y[half:]

# Run classifier
classifier = svm.SVC(kernel='linear', gamma=2, probability=True, random_state=0)
probas_ = classifier.fit(X_train, y_train).predict_proba(X_test)

# Compute ROC curve and area the curve
fpr, tpr, thresholds = roc_curve(y_test, probas_[:, 1])
roc_auc = auc(fpr, tpr)
print("Area under the ROC curve : %f" % roc_auc)

# Plot ROC curve
plt.clf()
plt.plot(fpr, tpr, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()

---

# Unsupervised Learning

Let us focus now on *unsupervised learning* and show you a set of tools  intended for the setting in which we have only a set of features $X_1,\ldots,X_p$  features measured on $n$ observations. We are not interested in prediction, because we do not have an associated response variable $Y$.  Rather, the goal is to discover interesting things about the measurements on $X_1,\ldots,X_p$.

In particular we will be interested in  *clustering*, a broad class of methods for discovering unknown subgroups in data.

## Principal Components Analysis

We looked at Principal components in Lecture 7 in the context of principal components regression. Recall when faced with a large set of correlated variables, principal components allow us to summarize this set with a smaller number of representative variables that collectively explain most of the variability in the original set. 

The principal component directions are  directions in feature space along which the original data are highly variable. These directions also define lines and subspaces that are as close as possible to the data cloud.  

PCA  refers to the process by which principal components are computed, and the subsequent use of these components in understanding the data.  PCA is is an unsupervised approach, since it involves only a set of features and no associated response.  It finds a low-dimensional representation of a data set that contains as much as possible of the variation. 

The idea is that each of the $n$ observations lives in $p$-dimensional space, but not all of these dimensions that are equally "interesting". PCA seeks a small number of dimensions that are as interesing as possible. Once we have computed the principal components, we can plot them against each other in order to produce low-dimensional views of the data.

It's easiest to visualize by looking at a two-dimensional dataset:

In [ ]:
X = np.dot(np.random.random(size=(2, 2)), np.random.normal(size=(2, 200))).T
plt.plot(X[:, 0], X[:, 1], 'og')
plt.axis('equal')

We can see that there is a definite trend in the data. What PCA seeks to do is to find the *Principal Axes* in the data, and explain how important those axes are in describing the data distribution:

In [ ]:
pca = PCA(n_components=2)
pca.fit(X)
print(pca.explained_variance_)
print(pca.components_)

In [ ]:
plt.plot(X[:, 0], X[:, 1], 'og', alpha=0.3)
plt.axis('equal')
for length, vector in zip(pca.explained_variance_, pca.components_):
    v = vector * 3 * np.sqrt(length)
    plt.plot([0, v[0]], [0, v[1]], '-k', lw=3)

Notice that one direction is very important, while the other direction is not.  This shows us that the second principal component could be *completely ignored* without much loss of information! Let's see what our data look like if we only keep 5% of the variance:

In [ ]:
clf = PCA(0.05)
X_trans = clf.fit_transform(X)
print(X.shape)
print(X_trans.shape)

By specifying that we want to throw away 95% of the variance, the data is now compressed by a factor of 50%! Let's see what the data look like after this compression:

In [ ]:
X_new = clf.inverse_transform(X_trans)
plt.plot(X[:, 0], X[:, 1], 'og', alpha=0.2)
plt.plot(X_new[:, 0], X_new[:, 1], 'og', alpha=0.8)
plt.axis('equal');

The light points are the original data, while the dark points are the projected version.  We see that after truncating 95% of the variance of this dataset and then reprojecting it, the "most important" features of the data are maintained, and we've compressed the data by 50%!

### Application of PCA to Digits

This might seem a bit strange in two dimensions, but the projection and dimensionality reduction can be extremely useful when visualizing high-dimensional data.  Let's take a quick look at the application of PCA to the digits data that comes with Scikit Learn:

In [ ]:
digits = load_digits()
X = digits.data
y = digits.target

# project from 64 to 2 dimensions
pca = PCA(2)
Xproj = pca.fit_transform(X)
print(X.shape)
print(Xproj.shape)

In [ ]:
plt.scatter(Xproj[:, 0], Xproj[:, 1], c=y)
plt.colorbar();

This gives us an idea of the relationship between the digits. Essentially, we have found the optimal rotation in 64-dimensional space that allows us to see the layout of the digits, *without reference* to the labels. But how much information have we thrown away?  We can figure this out by looking at the variance:

In [ ]:
pca = PCA(64).fit(X)
plt.semilogx(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance')

Here we see that our two-dimensional projection loses a lot of information (as measured by the explained variance) and that we'd need about 20 components to retain 90% of the variance.  Looking at this plot for a high-dimensional dataset can help you understand the level of redundancy present in multiple observations.

Note that scikit-learn contains many other unsupervised dimensionality reduction routines.

## Clustering Methods

When we cluster the observations of a data set, we seek to partition them into distinct groups so that the observations within each group are quite similar to each other, while observations in different groups are quite different from each other. Of course, to make this concrete, we must define what it means for two or more observations to be *similar* or *different*. Indeed, this is often a domain-specific consideration that must be made based on knowledge of the data being studied.

Since clustering is popular in many fields, there exist a great number of clustering methods. Today we focus on perhaps the two best-known clustering approaches: *K-means clustering* and *hierarchical clustering*.

## K-Means Clustering

K-means clustering is a simple and elegant approach for partitioning a data set into $K$ distinct, non-overlapping clusters. To perform K-means clustering, we must first specify the desired number of clusters $K$. Then the algorithm will assign each observation to exactly one of the $K$ clusters.

We begin by defining some notation. Let $C_1,\ldots,C_K$ denote sets containing the indices of the observations in each cluster. These sets satisfy two properties:
1. $C_1 \cup C_2 \cup \ldots \cup C_K = \{1,\ldots,n\}$ which says that each observation belongs to at least one of the $K$ clusters.
2. $C_k \cap C_{k'} = \emptyset $ for all $k \neq k'$ which says that the clusters are non-overlapping, no observation belongs to more than one cluster.

The idea behind K-means clustering is that a good clustering is one for which the *within-cluster variation* is as small as possible. The within-cluster variation for cluster $C_k$ is a measure $W(C_k)$ of the amount by which the observations within this cluster differ from each other. So K-means solves the minimisation problem:

$$
\min_{C_1,\ldots,C_K} \sum_{k=1}^{K} W(C_k).
$$

So what this says is that we want to  partition the observations into $K$ clusters such that the total within-cluster variation, summed over all $K$ clusters, is as small as possible.

This seems resonable but we need to define the within-cluster variation. The most common choice is the *squared Euclidean distance*:

$$
W(C_k) = \frac{1}{|C_k|} \sum_{i,i' \in C_k} \sum_{j=1}^{p} (x_{ij}-x_{i'j})^2,
$$

where $|C_k|$ | denotes the number of observations in the $k$th cluster. What this says is that the within-cluster variation for the $k$th cluster is the sum of all of the pairwise squared Euclidean distance between the observations in the $k$th cluster divided by the number of observations in the $k$th cluster.

To solve this is actually a very difficult since there are $K^n$ ways to partition $n$ observations into $K$ clusters. However there is a very efficient algorithm to find a local optimum to this optimisation problem. This is given by:
1. Randomly assign a number, from 1 to $K$ to each of the observations.
2. Iterate until the cluster assignments stop changing:
  * For each of the $K$ clusters, compute the cluster centroid. 
  * Assign each observation to the cluster whose centroid is closest.

This is guarenteed to decrease the value of the objective at each step. Because the K-means algorithm finds a local rather than a global optimum, the results obtained will depend on the initial (random) cluster assugbnebt of each observation in Step 1. For this reason, it is important to run the algorithm multiple times from different random initial configurations.  Then one selects the *best* solution.

The problem of selecting the $K$ is far from simple. And we address this below.

In [ ]:
X, y = make_blobs(n_samples=300, centers=4, cluster_std=0.60)
plt.scatter(X[:, 0], X[:, 1], s=50);

By eye, it is relatively easy to pick out the four clusters. If you were to perform an exhaustive search for the different segmentations of the data, however, the search space would be exponential in the number of points.

In [ ]:
est = KMeans(4)  # 4 clusters
est.fit(X)
y_kmeans = est.predict(X)
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans, s=50);

The algorithm identifies the four clusters of points in a manner very similar to what we would do by eye!

### Application of KMeans to Digits

For a closer-to-real-world example, let's again take a look at the digits data. Here we'll use KMeans to automatically cluster the data in 64 dimensions, and then look at the cluster centers to see what the algorithm has found.

In [ ]:
est = KMeans(n_clusters=10)
clusters = est.fit_predict(digits.data)
est.cluster_centers_.shape

We see ten clusters in 64 dimensions. Let's visualize each of these cluster centers to see what they represent:

In [ ]:
fig = plt.figure(figsize=(11, 5))
for i in range(10):
    ax = fig.add_subplot(2, 5, 1 + i, xticks=[], yticks=[])
    ax.imshow(est.cluster_centers_[i].reshape((8, 8)), cmap=plt.cm.binary)

We see that *even without the labels*, KMeans is able to find clusters whose means are recognizable digits (with apologies to the number 8).

For good measure, let's use our PCA visualization and look at the true cluster labels and K-means cluster labels:

In [ ]:
X = PCA(2).fit_transform(digits.data)

fig, ax = plt.subplots(1, 2, figsize=(11, 6))
ax[0].scatter(X[:, 0], X[:, 1], c=clusters)
ax[1].scatter(X[:, 0], X[:, 1], c=digits.target);

Though the colors are permuted, we see that in general (at least in a straightforward by-eye comparison) the KMeans clusters tend to reflect the true clustering.

As we have seen, to perform K-means clustering, we must decide how many clusters we expect in the data. The problem of selecting K is far from simple.

## Hierarchical Clustering

Hierarchical clustering is an alternative approach which does not require that we commit to a particular choice of $K$. Hierarchical clustering has an added advantage over K-means clustering in that it results in an attractive tree-based representation of the observations, called a *dendrogram*.

We first describe a bottom-up clustering approach. This is the most common type of hierarchical clustering, and refers to the fact that a dendrogram is built starting from the leaves and combining clusters up to the trunk. As we move up the tree some leaves begin to fuse into branches. These correspond to observations that are similar to each other. As we move higher up the tree, branches themselves fuse, either with leaves or other branches. The earlier a fuse occurs the more similar the groups of observations are.

![](../data/dendrogram.png)

So consider the above dendrogram obtained from hierarchically clustering nine observations. One can see that observations 5 and 7 are quite similar to each other, since they fuse at the lowest point on the dendrogram. 1 and 6 are similar too. However, it is tempting but incorrect to conclude from the figure that observations 9 and 2 are quite similar to each other on the basis that they are located near each other on the dendrogram.

In fact, based on the information contained in the dendrogram, observation 9 is no more similar to observation 2 than it is to observations 8, 5, and 7. We cannot draw conclusions about the similarity of two observations based on their proximity along the horizontal axis.  Rather, we draw conclusions about the similarity of two observations based on the location on the vertical axis.

How do we identify clusters? We  make a horizontal cut across the dendrogram and the sets of observations beneath the cut can be interpreted as clusters. The height of the cut plays the role of the $K$ in K-means.

### Hierarchical Clustering Algorithm

These dendrograms are obtained by a simple algorithm.  We begin by defining some sort of *dissimilarity* measure between each pair of observations. Most often, Euclidean distance is used. We then proceed iteratively.  Starting out at the bottom of the dendrogram, each of the $n$ observations is treated as its own cluster.  The two clusters that are most similar to each other are then fused so that there now are $n − 1$ clusters.   Next the two clusters that are most similar to each other are fused again, so that there now are $n − 2$ clusters.  We do this till all observations belong to a single cluster.

The concept of dissimilarity between a pair of observations needs to be extended to a pair of groups of observations. This extension is achieved by developing the notion of *linkage*, which defines the dissimilarity between two groups of observations.  The four most common types of linkage are:
* Complete: Maximal intercluster dissimilarity
* Single: Minimal intercluster dissimilarity
* Average: Mean intercluster dissimilarity
* Centroid: Dissimilarity between the centroid for cluster A and B.

Average and complete linkage are generally the prefered way to go as they yield more balanced dendrograms. 

###  Dissimilarity Measure

Up till now we have used Euclidean distance as a measure for dissimilarity. But sometimes other dissimilarity measures might be preferred. For example, *correlation-based distance* considers two observations to be similar  if their features are highly correlated, even though the observed values may be far apart in terms of Euclidean distance.

The choice of dissimilarity measure is very important, as it has a strong effect on the resulting dendrogram. In general, careful attention should be paid to the type of data being clustered and the scientific question at hand.

## Issues in Clustering

Clustering can be a very useful tool for data analysis in the unsupervised setting. However, there are a number of issues that arise in performing clustering.

In order to perform clustering, some decisions must be made:
* Should the observations or features first be standardized in some way? For instance, maybe the variables should be centered to have mean zero and scaled to have standard deviation one.
* In the case of hierarchical clustering, what dissimilarity, linkage should be used and where should we cut?
* In the case of K-means clustering, how many clusters should we look for in the data?

With these methods, there is no single right answer---any solution that exposes some interesting aspects of the data should be considered.

### Validating the Clusters 

We really want to know whether the clusters that have been found represent true subgroups in the data, or whether they are simply a result of *clustering the noise*. This is a hard question to answer. There exist a number of techniques for assigning a p-value to a cluster in order to assess whether there is more evidence for the cluster than one would expect due to chance. 